In [64]:
##Run the below block to initialize all the dependencies
import requests
from pprint import pprint
import pandas as pd
from pandas import ExcelWriter
import io, time, json
import warnings
warnings.filterwarnings("ignore")

In [65]:
##The below block of code is used to read the API Key required for extracting data from Yelp fusion API.

def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

def read_api_key_zomato(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    with open('api_key_zomato.txt', 'r') as f:
        return f.read().replace('\n','')

In [66]:
##The function below is used to retrieve all the restaurants from the YELP API.

def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    
    offsetno=0
    total =0
    output=[]
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    while (offsetno < 1000):
        params = { #parameters are case sensitive!
            "location": query,
            "limit": 20,
            "offset": offsetno,
            "categories": "restaurants"
            
        }
    
        response = requests.get('https://api.yelp.com/v3/businesses/search',
                            headers=headers, params=params)
        result = response.json()
        try:
            output.append( result['businesses'])
        except KeyError:
            print(result)
            return output
        offsetno =offsetno + 20
        total = result['total']
        #print (offsetno)
        #print (len(output))
    return(output)

api_key = read_api_key('api_key.txt')
data = all_restaurants(api_key, 'Miami')
new_data = {}
new_data["name"]=[]
new_data["rating"]=[]
new_data["price"]=[]
new_data["review_count"]=[]
new_data["url"]=[]
new_data["category"]=[]
new_data["zip"]=[]
new_data["Source"]=[]
alldata=[]
for i in data:
    for each in i:
        new_data["name"].append(each["name"])
        new_data["rating"].append(each["rating"])
        new_data["url"].append(each["url"])
        try:
            new_data["price"].append(each["price"])
        except KeyError:
            new_data["price"].append(None)

        new_data["review_count"].append(each["review_count"])
        new_data["category"].append(each["categories"][0]["title"])
        new_data["zip"].append(each["location"]["zip_code"])
        new_data["Source"].append('Yelp')
        alldata.append(each)
writer = pd.ExcelWriter('../DataFiles/Final_Data.xlsx' , engine='xlsxwriter')
yelp_raw = pd.DataFrame(alldata)
yelp_clean = pd.DataFrame(new_data)

In [67]:
##The below code needs to be executed to clean the TripAdvisor CSV file and create a raw and clean dataset out of it.

data = pd.read_csv('../DataFiles/TripAdvisor_Data.csv')
trip_advisor_raw = data[data["City"]=="Miami"]
trip_advisor_clean = cleaned_data.loc[:,["Restaurant ID", "Restaurant URL","Name","Address","Ranking","Total Review"]]

In [68]:
##Below block of code is used to extract the data from the Zomato API.

api_key_zomato = read_api_key_zomato('api_key_zomato.txt')

headers = {'Content-Type': 'application/json', 'user-key': api_key_zomato}
offset = 0
restaurantNameList = []
ratingList = []
ratingTextList = []
votesList = []
totalData = []
urlList = []
addressList = []
sourceList = []
while offset < 100:
    url = 'https://developers.zomato.com/api/v2.1/search?entity_id=291&entity_type=city&start='+str(offset)+'&count=20'
    Response = requests.get(url , headers=headers)
    offset += 20
    Response_Json = Response.json()
    Array_length = Response_Json['restaurants']
    for i in range(len(Array_length)):
        totalData.append(Response_Json['restaurants'][i])
        restaurantNameList.append(Response_Json['restaurants'][i]['restaurant']['name'])
        ratingList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['aggregate_rating'])
        ratingTextList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['rating_text'])
        votesList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['votes'])
        urlList.append(Response_Json['restaurants'][i]['restaurant']['url'])
        addressList.append(Response_Json['restaurants'][i]['restaurant']['location']['address'])
        sourceList.append('Zomato')
zomato_raw = pd.DataFrame({'AllData':totalData})
zomato_clean = pd.DataFrame({'Restaurant Name':restaurantNameList,'Rating':ratingList, 'Rating Text':ratingTextList, 'Votes':votesList, 'Url':urlList, 'Address': addressList, 'Source': sourceList})

In [69]:
##Run the below block to create the excel file with the collated data

writer = pd.ExcelWriter('../DataFiles/Final_Data.xlsx' , engine='xlsxwriter')

yelp_raw.to_excel(writer,'YelpRaw',index=False)
yelp_clean.to_excel(writer,'YelpClean',index=False)
trip_advisor_raw.to_excel(writer, sheet_name = ' TripAdvisor_raw')
trip_advisor_clean.to_excel(writer, sheet_name = 'TripAdvisor_clean')
zomato_raw.to_excel(writer,sheet_name = 'Zomato_raw')
zomato_clean.to_excel(writer,sheet_name = 'Zomato_clean')

writer.save()
writer.close()